In [7]:
#Подключаем нужные библиотеки
import numpy as np
import pandas as pd

In [8]:
#Загружаем все содержимое файлов
movies = pd.read_csv('movies.csv', delimiter=',')
ratings = pd.read_csv('ratings.csv', delimiter=',')

In [9]:
#Удаляем неиспользуемый столбец времени и "странных" пользователей 
#"Странные" пользователи - те, кто смотрели фильмы ~15-20 лет подряд. 

ratings=ratings.drop('timestamp' ,axis=1)

ratings=ratings.drop(ratings[ratings['userId']==72315].index) #посмотрел больше 32000 фильмов
ratings=ratings.drop(ratings[ratings['userId']==33844].index) #посмотрели больше 7500 фильмов
ratings=ratings.drop(ratings[ratings['userId']==80974].index) 
ratings=ratings.drop(ratings[ratings['userId']==137293].index)

In [10]:
# Добавляем в ratings колонку с названием ЖАНРА фильма
ratings = pd.merge(ratings, movies[['movieId','genres']], how='left', on='movieId')

# Добавляем в ratings колонку UserMovieCount с общим количеством фильмов, просмотренных пользователем 
# UserMovieCount
ratings['UserMovieCount']=ratings[['userId','movieId']].groupby('userId')['userId'].transform('count')

# Удаляем из ratings пользователей, которые 13 лет подряд смотрели кажды день новый фильм
ratings = ratings.drop(ratings[ratings['UserMovieCount']>5000].index)

# Добавляем в ratings колонку UserGenresCount с количеством просмотренных пользователем фильмов ОПРЕДЕЛЕННГО ЖАНРА 
# UserGenresCount
ratings['UserGenresCount'] = ratings[['userId','genres','rating']].groupby(['userId','genres'])['genres'].transform('count')

# Добавляем в movies колонку UserIdCount с количеством пользователей посмотревших фильмов
# UserIdCount
watchedcount=ratings[['movieId','userId']].groupby(['movieId']).count()
movies=pd.merge(movies,watchedcount, how='left', on='movieId', validate='one_to_one')
del(watchedcount)
movies=movies.rename(columns={"userId": "UserIdCount"})

In [11]:
# Расчитываем по формуле ВЕС (ценность или стоимость) оценки, которую пользователь поставил фильму
# и записываем ее в колонку RatingWeight в ratings
#     Чем больше пользователь посмотрел фильмов, тем больше доверия его оценке, тем больше RatingWeight
#     Чем больше пользователь посмотрел фильмов данного жанра, тем больше доверия его оценке, тем больше RatingWeight

ratings['RatingWeight']=np.log(ratings['UserGenresCount']+0.2)*np.log(ratings['UserMovieCount']+0.2)

#Добавляем в ratings колонку RatingProdWeight - произведение Оценки фильма (rating) на ВесОЦенки (RatingWeight) 
ratings['RatingProdWeight']=ratings['rating']/5*ratings['RatingWeight']

#  В таблицу movies добавляем колонку MovieRating - рейтинг фильма, сумма всех выставленных ему оценок
moviratings=ratings[['movieId','RatingProdWeight']].groupby("movieId").sum()
movies=pd.merge(movies,moviratings, how='left', on='movieId', validate='one_to_one')
movies=movies.rename(columns={"RatingProdWeight": "MovieRating"})

In [12]:
ratings

,userId,movieId,rating,genres,UserMovieCount,UserGenresCount,RatingWeight,RatingProdWeight
0,1,296,5.0,Comedy|Crime|Drama|Thriller,70,1,0.775112,0.775112
1,1,306,3.5,Drama,70,22,13.179572,9.225700
2,1,307,5.0,Drama,70,22,13.179572,13.179572
3,1,665,5.0,Comedy|Drama|War,70,2,3.352007,3.352007
4,1,899,3.5,Comedy|Musical|Romance,70,1,0.775112,0.542579
...,...,...,...,...,...,...,...,...
24941878,162541,50872,4.5,Animation|Children|Drama,182,1,0.949003,0.854103
24941879,162541,55768,2.5,Animation|Comedy,182,1,0.949003,0.474501
24941880,162541,56176,2.0,Children|Comedy,182,1,0.949003,0.379601
24941881,162541,58559,4.0,Action|Crime|Drama|IMAX,182,1,0.949003,0.759202


In [13]:
movies

,movieId,title,genres,UserIdCount,MovieRating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,57293.0,177904.260944
1,2,Jumanji (1995),Adventure|Children|Fantasy,24212.0,73073.552839
2,3,Grumpier Old Men (1995),Comedy|Romance,11791.0,81263.927210
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2518.0,18505.970321
4,5,Father of the Bride Part II (1995),Comedy,11700.0,95245.139934
...,...,...,...,...,...
62418,209157,We (2018),Drama,1.0,1.418073
62419,209159,Window of the Soul (2001),Documentary,1.0,22.668347
62420,209163,Bad Poems (2018),Comedy|Drama,1.0,24.758222
62421,209169,A Girl Thing (2001),(no genres listed),1.0,2.836147


In [14]:
####   Код для формирования рекомендаций для кокретного пользователя (с номером userID)

# Для какогото пользователя userID делаем таблицу userpref с просмотренными Жанрами 
# и считаем рейтинг  Жанров пользователя UserGenresRating. 
# Умножаем среднюю оценку фильмов каждого жанра на количество просмотренных фильмов данного жанра
# и делим на общее число просмотренных фильмов

userID=2567  # это ID пользователя, для которого будем делать рекомендации
NumberGenres = 4 #это число жанров пользователя, для которых будем рекомендоввать фильмы

UserMovieCount=ratings[ratings['userId']==userID]['UserMovieCount'].iloc[0]
userpref=ratings[ratings['userId']==userID].groupby(['genres']).agg({'rating':'mean','userId':'count'})
userpref=userpref.rename(columns={"rating":"UserGenresMeanRating", "userId": "UserGenresCount"})
userpref['UserGenresRating']=(userpref['UserGenresCount']/UserMovieCount)*userpref['UserGenresMeanRating']

#Выбираем сколько-то NumberGenres первых по значению UserGenresRating жанров - Самые Любимые жанры 
userpref=userpref.sort_values(by=['UserGenresRating'], ascending=False)[:NumberGenres]
userpref.head(20)

,UserGenresMeanRating,UserGenresCount,UserGenresRating
genres,,,
Drama,3.835821,67,0.753666
Comedy|Drama,3.821429,14,0.156891
Drama|Romance,3.500000,14,0.143695
Crime|Drama,4.000000,12,0.140762


In [15]:
# Находим ВСЕ фильмы для любимых жанров пользователя, которые он еще не видел


# Сначала находим все фильмы, которые пользователь смотрел и записываем их в таблицу useralreadysee
useralreadysee=ratings[['movieId']][ratings['userId']==userID].groupby(['movieId'], as_index=False).first()

# Теперь в таблицу usernotsee записываем фильмы всех (вообще всех) жанров, которые пользователь еще не смотрел
usernotsee=movies[pd.merge(movies,useralreadysee, how='left', on='movieId', indicator=True)['_merge']=='left_only']

# Cоздаем таблицу advisemovie с не просмотренными фильмами Любимых жанров пользователя (которые берем из таблицы userpref)
# Для каждого жанра фильмы отсортированы в порядке возрастания MovieRating
advisemovie=pd.merge(userpref,usernotsee, how='left', on = 'genres').sort_values(by=['UserGenresRating','MovieRating'], ascending=False)

# Выбираем в каждом Любимом жанре  сколько-то CountInGenres рекомендованных фильмов
CountInGenres=4 # это количество фильмов, которое будет рекомендовано в каждом жанре
groupedadvise=advisemovie.groupby(by='genres', sort=False)
groupedadvise.apply(lambda subf: subf[:CountInGenres])[['genres','UserGenresRating','movieId','title','MovieRating']]

genres  UserGenresRating  movieId  \
genres                                                          
Drama         227            Drama          0.753666     1246   
              224            Drama          0.753666     1225   
              658            Drama          0.753666     4022   
              636            Drama          0.753666     3897   
Comedy|Drama  9015    Comedy|Drama          0.156891      500   
              9232    Comedy|Drama          0.156891     4979   
              9109    Comedy|Drama          0.156891     2395   
              9074    Comedy|Drama          0.156891     1641   
Drama|Romance 11443  Drama|Romance          0.143695     1704   
              11444  Drama|Romance          0.143695     1721   
              11559  Drama|Romance          0.143695     4995   
              11411  Drama|Romance          0.143695      912   
Crime|Drama   13487    Crime|Drama          0.140762      858   
              13494    Crime|Drama          0.140762     1221   
              13493    Crime|Drama          0.140762     1213   
              13512    Crime|Drama          0.140762     3147   

                                              title    MovieRating  
genres                                                              
Drama         227         Dead Poets Society (1989)  329169.015744  
              224                    Amadeus (1984)  300036.207701  
              658                  Cast Away (2000)  293790.681210  
              636              Almost Famous (2000)  255875.456152  
Comedy|Drama  9015            Mrs. Doubtfire (1993)  211544.943927  
              9232     Royal Tenenbaums, The (2001)  169428.263252  
              9109                  Rushmore (1998)  160770.040427  
              9074           Full Monty, The (1997)  156047.790593  
Drama|Romance 11443        Good Will Hunting (1997)  346066.074614  
              11444                  Titanic (1997)  287709.637440  
              11559        Beautiful Mind, A (2001)  275832.582097  
              11411               Casablanca (1942)  260145.955023  
Crime|Drama   13487           Godfather, The (1972)  427372.311134  
              13494  Godfather: Part II, The (1974)  316505.825886  
              13493               Goodfellas (1990)  311913.788026  
              13512          Green Mile, The (1999)  257878.106070